In [15]:
import os
import pickle as pkl
import numpy as np

In [4]:
datadir1='./tensorRGflow-brucelyu/out/gilt_hotrg22_flow/eps6e-05_chi16/'
datadir2='./tensorRGflow-brucelyu/analysisCodes/data/gilt_hotrg22_flow/eps6e-05_chi16/'

os.listdir(datadir2)

['As', 'otherTs.pkl']

In [8]:
with open(datadir2+'/otherTs.pkl','rb') as f:
    Anorm,isomlist,RABslist,RABshlist=pkl.load(f)

In [13]:
type(isomlist[0][0])

numpy.ndarray

In [ ]:

def halfHOTRG(B, A, chi, direction = "v", verbose = True, cg_eps = 1e-6,
              isjax = False, evenTrunc = False):
    """
    Perform half of the HOTRG coarse graining
    """

    if direction == "v":
        if verbose:
            print("Coarse graining in y direction...")
        # determine the isometry that squeezes legs in y direction
        if not isjax:
            w, dw, SP1err = yProjectorAB(B, A, chi, cg_eps,
                                         evenTrunc = evenTrunc)
        else:
            w, dw, SP1err = yProjectorAB(B, A, chi, isjax = True)
        if verbose:
            print("Spectrum of B @ A is:")
            dwarr = dw / dw.max()
            dwarr = np.abs(dwarr.to_ndarray())
            dwarr = - np.sort(-dwarr)
            print(dwarr[:10])
            print("Bound dimension in y direction is {:d}".format(len(dw)))
            print("Truncation error would be {:.3e}".format(SP1err))
            print("Perform contraction along y direction...")
        # contraction
        if not isjax:
            Ap = ncon([B, A, w.conjugate(), w],
                  [[1,4,-3,2],[3,5,2,-4],[1,3,-1],[4,5,-2]])
        else:
            Ap = jncon([B, A, w, w],
                  [[1,4,-3,2],[3,5,2,-4],[1,3,-1],[4,5,-2]])
        if verbose:
            print("Contraction in y direction finished!\n")
    elif direction == "h":
        if verbose:
            print("Coarse graining in x direction...")
        if not isjax:
            w,dw,SP2err = xProjectorAB(B, A, chi, cg_eps,
                                       evenTrunc = evenTrunc)
        else:
            w,dw,SP2err = xProjectorAB(B, A, chi, isjax = True)
        if verbose:
            print("Spectrum of B @ A is:")
            dwarr = dw / dw.max()
            dwarr = np.abs(dwarr.to_ndarray())
            dwarr = - np.sort(-dwarr)
            print(dwarr[:10])
            print("Bound dimension in y direction is {:d}".format(len(dw)))
            print("Truncation error would be {:.3e}".format(SP2err))
            print("Perform contraction along x direction...")
        # contraction
        if not isjax:
            Ap = ncon([B, A, w.conjugate(), w],
                   [[-1,2,1,4],[2,-2,3,5],[1,3,-3],[4,5,-4]])
        else:
            Ap = jncon([B, A, w, w],
                   [[-1,2,1,4],[2,-2,3,5],[1,3,-3],[4,5,-4]])
        if verbose:
            print("Contraction in x direction finished!\n")
    else:
        raise ValueError("variable direction can only choose between h and v.")
    return Ap, w


In [ ]:
## The isometric tensors are given, contract them with original tensors

def doHalfHOTRGknownWV(B, A, w, direction = "v"):
    B = convertAbeBack(B)
    A = convertAbeBack(A)
    if direction == "v":
        Ap = jncon([B, A, w.conjugate(), w],
                  [[1,4,-3,2],[3,5,2,-4],[1,3,-1],[4,5,-2]])
    elif direction == "h":
        Ap = jncon([B, A, w.conjugate(), w],
                   [[-1,2,1,4],[2,-2,3,5],[1,3,-3],[4,5,-4]])
    else:
        raise ValueError("Variable direction can only choose between h and v.")
    return Ap


In [16]:
np.zeros(5).conj()

array([0., 0., 0., 0., 0.])

In [ ]:
def diffGiltHOTRG(A, Anorm, isom, RABs, RABsh, scaleN = 20,
               isom_corr = False):
    """
    Similar as diffRGnew, but designed for Gilt-HOTRG-imp version
    """
    # define the invariant tensor where the magnetitute is properly taken care of
    Ainv = Anorm**(-1/3) * A
    # read of isometries and R matrices used in Gilt
    w, v = isom
    RAl, RAr, RBl, RBr = RABs[2:]
    RAlh, RArh, RBlh, RBrh = RABsh[2:]
    # convert everything to numpy.array for consistent, since we will
    # fall back to ordinary tensor multiplication in the calculation here
    Ainv = convertAbeBack(Ainv)
    N1, N2, N3, N4 = Ainv.shape
    w = convertAbeBack(w)
    v = convertAbeBack(v)
    RAl = convertAbeBack(RAl)
    RAr = convertAbeBack(RAr)
    RBl = convertAbeBack(RBl)
    RBr = convertAbeBack(RBr)

    RAlh = convertAbeBack(RAlh)
    RArh = convertAbeBack(RArh)
    RBlh = convertAbeBack(RBlh)
    RBrh = convertAbeBack(RBrh)

    # define the RG equation
    def equationRG(psiA):
        Aorg = psiA.reshape(N1,N2,N3,N4)
        # Gilt before y-contraction
        Ap = jncon([Aorg, RAl, RAr], [[1, 2, -3, -4], [1,-1], [2,-2]])
        Bp = jncon([Aorg, RBl, RBr], [[1, 2, -3, -4], [1,-1], [2,-2]])
        # perform HOTRG y-contraction
        if not isom_corr:
            Ap = doHalfHOTRGknownWV(Bp, Ap, w, direction = "v")
        else:
            chiH = w.shape[2]
            Ap = halfHOTRG(Bp, Ap, chiH, direction = "v", verbose = False,
                           isjax = True)[0]
        # Gilt before x-contraction
        App = jncon([Ap, RAlh, RArh], [[-1,-2,3,4], [4,-4], [3,-3]])
        Bpp = jncon([Ap, RBlh, RBrh], [[-1,-2,3,4], [4,-4], [3,-3]])
        # perform HOTRG x-contraction
        if not isom_corr:
            Ap = doHalfHOTRGknownWV(Bpp, App, v, direction = "h")
        else:
            chiV = v.shape[2]
            Ap = halfHOTRG(Bpp, App, chiV, direction = "h", verbose = False,
                           isjax = True)[0]
        psiAp = Ap.reshape(N1 * N2 * N3 * N4)
        return psiAp
    # linearlized the RG equation to get response matrix
    dimA = N1 * N2 * N3 * N4
    psiA = Ainv.reshape(dimA)
    psiAp, responseMat = jax.linearize(equationRG, psiA)
    # calculate its eigenvalues
    RGhyperM = LinearOperator((dimA,dimA), matvec = responseMat)
    dtemp = np.sort(abs(eigs(RGhyperM, k=scaleN,
                    which='LM', return_eigenvectors=False)))
    dtemp = dtemp[::-1]
    # calculate scaling dimensions
    scDims = -np.log2(abs(dtemp/dtemp[0]))
    return scDims